In [1]:
import tensorflow as tf
import string
import requests

In [2]:
response=requests.get('https://www.gutenberg.org/files/1952/1952-0.txt')

In [5]:
response.text
data=response.text.split('\n')
data[29]

'It is very seldom that mere ordinary people like John and myself secure'

In [6]:
data=data[29:]

In [7]:
len(data)

1169

In [8]:
data=" ".join(data)

In [9]:
def clean_text(doc):
    tokens=doc.split()
    table=str.maketrans('','',string.punctuation)
    tokens=[w.translate(table) for w in tokens]
    tokens=[word for word in tokens if word.isalpha()]
    tokens=[word.lower() for word in tokens]
    return tokens

In [10]:
tokens=clean_text(data)

In [11]:
tokens[:50]
len(tokens)

8723

In [12]:
length=60+1
lines=[]
for i in range(length,len(tokens)):
    seq=tokens[i-length:i]
    line=' '.join(seq)
    lines.append(line)

In [13]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequence=tokenizer.texts_to_sequences(lines)

In [90]:
sequence

[[7,
  8,
  62,
  1687,
  10,
  1685,
  1684,
  162,
  65,
  32,
  2,
  161,
  742,
  1682,
  1681,
  14,
  1,
  1679,
  6,
  1678,
  1677,
  6,
  1676,
  741,
  3,
  50,
  231,
  6,
  1675,
  202,
  2,
  288,
  1,
  1674,
  4,
  1672,
  10,
  50,
  25,
  1670,
  118,
  77,
  4,
  1665,
  287,
  3,
  71,
  1664,
  1663,
  10,
  34,
  8,
  159,
  484,
  43,
  7,
  286,
  158,
  157,
  7,
  25],
 [8,
  62,
  1687,
  10,
  1685,
  1684,
  162,
  65,
  32,
  2,
  161,
  742,
  1682,
  1681,
  14,
  1,
  1679,
  6,
  1678,
  1677,
  6,
  1676,
  741,
  3,
  50,
  231,
  6,
  1675,
  202,
  2,
  288,
  1,
  1674,
  4,
  1672,
  10,
  50,
  25,
  1670,
  118,
  77,
  4,
  1665,
  287,
  3,
  71,
  1664,
  1663,
  10,
  34,
  8,
  159,
  484,
  43,
  7,
  286,
  158,
  157,
  7,
  25,
  163],
 [62,
  1687,
  10,
  1685,
  1684,
  162,
  65,
  32,
  2,
  161,
  742,
  1682,
  1681,
  14,
  1,
  1679,
  6,
  1678,
  1677,
  6,
  1676,
  741,
  3,
  50,
  231,
  6,
  1675,
  202,
  2,
  288,
  1,

In [15]:
sequences=np.array(sequence)

In [89]:
sequences

array([[   7,    8,   62, ...,  157,    7,   25],
       [   8,   62, 1687, ...,    7,   25,  163],
       [  62, 1687,   10, ...,   25,  163,   19],
       ...,
       [1661,  269,  234, ..., 1686,    5,  743],
       [ 269,  234,    9, ...,    5,  743,   43],
       [ 234,    9,  285, ...,  743,   43,  370]])

In [16]:
X,y=sequences[:,:-1],sequences[:,-1]

In [17]:
X[0],y[0]

(array([   7,    8,   62, 1687,   10, 1685, 1684,  162,   65,   32,    2,
         161,  742, 1682, 1681,   14,    1, 1679,    6, 1678, 1677,    6,
        1676,  741,    3,   50,  231,    6, 1675,  202,    2,  288,    1,
        1674,    4, 1672,   10,   50,   25, 1670,  118,   77,    4, 1665,
         287,    3,   71, 1664, 1663,   10,   34,    8,  159,  484,   43,
           7,  286,  158,  157,    7]),
 25)

In [18]:
vocab_size=len(tokenizer.word_index)+1

In [91]:
vocab_size

1688

In [19]:
y=to_categorical(y,num_classes=vocab_size)

In [93]:
y.shape

(8662, 1688)

In [20]:
seq_length=X.shape[1]

In [21]:
model =Sequential()
model.add(Embedding(vocab_size,60,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(y.shape[1],activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(X,y,batch_size=256,epochs=300)

Epoch 1/300
34/34 [==============================] - 11s 328ms/step - loss: 6.0538 - accuracy: 0.0421
Epoch 2/300
34/34 [==============================] - 11s 326ms/step - loss: 6.0173 - accuracy: 0.0439
Epoch 3/300
34/34 [==============================] - 11s 322ms/step - loss: 6.0041 - accuracy: 0.0461
Epoch 4/300
34/34 [==============================] - 11s 319ms/step - loss: 6.0004 - accuracy: 0.0464
Epoch 5/300
34/34 [==============================] - 11s 324ms/step - loss: 5.9957 - accuracy: 0.0466
Epoch 6/300
34/34 [==============================] - 11s 324ms/step - loss: 5.9947 - accuracy: 0.0424
Epoch 7/300
34/34 [==============================] - 11s 318ms/step - loss: 5.9904 - accuracy: 0.0464
Epoch 8/300
34/34 [==============================] - 11s 322ms/step - loss: 5.9844 - accuracy: 0.0464
Epoch 9/300
34/34 [==============================] - 11s 335ms/step - loss: 5.9472 - accuracy: 0.0470
Epoch 10/300
34/34 [==============================] - 11s 322ms/step - loss: 5.853

34/34 [==============================] - 11s 326ms/step - loss: 3.6934 - accuracy: 0.2381
Epoch 81/300
34/34 [==============================] - 11s 324ms/step - loss: 3.6779 - accuracy: 0.2408
Epoch 82/300
34/34 [==============================] - 11s 321ms/step - loss: 3.6534 - accuracy: 0.2405
Epoch 83/300
34/34 [==============================] - 11s 325ms/step - loss: 3.6269 - accuracy: 0.2479
Epoch 84/300
34/34 [==============================] - 11s 329ms/step - loss: 3.5998 - accuracy: 0.2495
Epoch 85/300
34/34 [==============================] - 11s 320ms/step - loss: 3.5750 - accuracy: 0.2551
Epoch 86/300
34/34 [==============================] - 11s 324ms/step - loss: 3.5496 - accuracy: 0.2591
Epoch 87/300
34/34 [==============================] - 12s 340ms/step - loss: 3.5312 - accuracy: 0.2607
Epoch 88/300
34/34 [==============================] - 12s 357ms/step - loss: 3.5081 - accuracy: 0.2646
Epoch 89/300
34/34 [==============================] - 12s 360ms/step - loss: 3.4870 - 

34/34 [==============================] - 11s 322ms/step - loss: 2.2625 - accuracy: 0.5174
Epoch 160/300
34/34 [==============================] - 11s 328ms/step - loss: 2.2512 - accuracy: 0.5192
Epoch 161/300
34/34 [==============================] - 11s 331ms/step - loss: 2.2338 - accuracy: 0.5222
Epoch 162/300
34/34 [==============================] - 11s 326ms/step - loss: 2.2243 - accuracy: 0.5241
Epoch 163/300
34/34 [==============================] - 11s 320ms/step - loss: 2.2191 - accuracy: 0.5236
Epoch 164/300
34/34 [==============================] - 11s 328ms/step - loss: 2.2048 - accuracy: 0.5285
Epoch 165/300
34/34 [==============================] - 12s 365ms/step - loss: 2.1883 - accuracy: 0.5287
Epoch 166/300
34/34 [==============================] - 11s 320ms/step - loss: 2.1787 - accuracy: 0.5328
Epoch 167/300
34/34 [==============================] - 11s 316ms/step - loss: 2.1576 - accuracy: 0.5366
Epoch 168/300
34/34 [==============================] - 11s 329ms/step - loss: 

34/34 [==============================] - 11s 323ms/step - loss: 1.4265 - accuracy: 0.6981
Epoch 238/300
34/34 [==============================] - 11s 327ms/step - loss: 1.4287 - accuracy: 0.6930
Epoch 239/300
34/34 [==============================] - 11s 329ms/step - loss: 1.4239 - accuracy: 0.6953
Epoch 240/300
34/34 [==============================] - 11s 321ms/step - loss: 1.3926 - accuracy: 0.7023
Epoch 241/300
34/34 [==============================] - 11s 331ms/step - loss: 1.3890 - accuracy: 0.7025
Epoch 242/300
34/34 [==============================] - 11s 328ms/step - loss: 1.4448 - accuracy: 0.6864
Epoch 243/300
34/34 [==============================] - 11s 326ms/step - loss: 1.4118 - accuracy: 0.6995
Epoch 244/300
34/34 [==============================] - 11s 321ms/step - loss: 1.3786 - accuracy: 0.7071
Epoch 245/300
34/34 [==============================] - 11s 327ms/step - loss: 1.3657 - accuracy: 0.7090
Epoch 246/300
34/34 [==============================] - 11s 328ms/step - loss: 

In [79]:
seed_text=lines[1000]
# test_text="And indeed bad as his clothes were and coarsely as he spoke, he had none of the appearance of a man who sailed before the mast, but seemed like a mate or skipper accustomed to be obeyed or to strike. The man who came with the barrow told us the mail had set him down the morning before at the royal George, that he had inquired what inns there were along"

In [69]:
# test_text=clean_text(test_text)
# test_text

In [70]:
# lines_test=[]
# for i in range(length,len(test_text)):
#     seq=test_text[i-length:i]
#     line=' '.join(seq)
#     lines_test.append(line)

In [71]:
# lines_test[0]
# # len(test_text)

In [72]:
# X.shape[1]

In [86]:
def generate_text_seq(model, tokenizer,text_seq_length,seed_text,n_words):
    text=[]
    for _ in range(n_words):
        encoded= tokenizer.texts_to_sequences([seed_text])[0]
        encoded=pad_sequences([encoded],maxlen=text_seq_length,truncating='pre')
        
        y_predict=model.predict_classes(encoded)
        
        predicted_word=''
        for word,index in tokenizer.word_index.items():
            if index==y_predict:
                predicted_word=word
                break
        seed_text=seed_text + ' '+predicted_word
        text.append(predicted_word)
    return ' '.join(text)

        
        

In [87]:
generate_text_seq(model,tokenizer,seq_length,seed_text,100)

'already nobody would believe what an effort it is why you to believe out to the front pattern it is very lack of superstition and to have been rubbed up and there is fortunate mary you you may choose to feel it that i want to have to get her and makes me dizzy at myself he the daytime and is very stay of strength it is always believe i sleep her for that does to sleep out to the pattern is fairly many than i tried it is a week more in the stairs i turned so much as'

# for rough work

In [88]:
encoded= tokenizer.texts_to_sequences([seed_text])[0]
encoded
encoded=pad_sequences([encoded],maxlen=seq_length,truncating='pre')
encoded
y_predict=model.predict_classes(encoded)
y_predict
predicted_word=''
for word,index in tokenizer.word_index.items():
    if index==y_predict:
        predicted_word=word
        break
predicted_word


'already'